In [1]:
import cv2

In [2]:
cv2.__version__


'4.1.2'

## Імпортувати картинку

In [3]:
img = cv2.imread('table_objects.jpg', 0)

In [4]:
cv2.namedWindow('image', cv2.WINDOW_NORMAL)
cv2.imshow('image', img)
cv2.waitKey(0)
cv2.destroyAllWindows()

## Відео

In [5]:
cap = cv2.VideoCapture(0)

In [6]:
while(True):
    ret, frame = cap.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    cv2.imshow('frame',gray)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cv2.waitKey(0)
cv2.destroyAllWindows()
cap.release()

 ## Малювання геометричних форм

+ Лінія.
+ Коло.
+ Квадрат.
+ Елліпс.
+ Текст.


In [7]:
import numpy as np
import cv2

In [8]:
#чорна картинка
img = np.zeros((512, 512,3), np.uint8)

### Лінія

In [9]:
# Діагональна синя лінія
img = cv2.line(img, (100, 0), (511, 511), (255, 0, 0), 5)

In [10]:
cv2.imshow('frame', img)
cv2.waitKey(0)
cv2.destroyAllWindows()

### Квадрат

In [11]:
img = cv2.rectangle(img, (100, 100), (411, 311), (0, 255, 255), 5)

In [12]:
cv2.imshow('frame', img)
cv2.waitKey(0)
cv2.destroyAllWindows()

## Елліпс

In [13]:
img = cv2.ellipse(img, (256, 256), (100, 50), 0, 0, 270, 255, -1)

In [14]:
cv2.imshow('frame', img)
cv2.waitKey(0)
cv2.destroyAllWindows()

## Полігон

In [15]:
pts = np.array([[10, 5], [20, 30], [70, 20],[50, 10]], np.int32)
pts = pts.reshape((-1, 1, 2))
img = cv2.polylines(img, [pts], True, (0, 255, 255))


In [16]:
cv2.imshow('frame', img)
cv2.waitKey(0)
cv2.destroyAllWindows()

## Текст

In [17]:
font = cv2.FONT_HERSHEY_SIMPLEX
cv2.putText(img, 'OpenCV', (10, 500), font, 4, (255, 255, 255), 2, cv2.LINE_AA)

array([[[  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0],
        ...,
        [  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0]],

       [[  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0],
        ...,
        [  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0]],

       [[  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0],
        ...,
        [  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0]],

       ...,

       [[  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0],
        ...,
        [255,   0,   0],
        [255,   0,   0],
        [255,   0,   0]],

       [[  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0],
        ...,
        [255,   0,   0],
        [255,   0,   0],
        [255,   0,   0]],

       [[  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0],
        ...,
        [255,   0,   0],
        [255,   0,   0],
        [255,   0,   0]]

In [18]:
cv2.imshow('frame', img)
cv2.waitKey(0)
cv2.destroyAllWindows()

## Виділення рухомих обєктів

### Спосіб 1: createBackgroundSubtractorKNN

In [32]:
backSub = cv2.createBackgroundSubtractorKNN(1000, 10, True)

capture = cv2.VideoCapture(0)
if not capture.isOpened:
    print('Unable to open: Передня камера' )
    exit(0)

while True:
    ret, frame = capture.read()
    if frame is None:
        break

    fgMask = backSub.apply(frame)
    
    cv2.rectangle(frame, (10, 2), (100, 20), (255, 255, 255), -1)
    cv2.putText(frame, str(capture.get(cv2.CAP_PROP_POS_FRAMES)), (15, 15),
               cv2.FONT_HERSHEY_SIMPLEX, 0.5 , (0, 0, 0))

    cv2.imshow('Frame', frame)
    cv2.imshow('FG Mask', fgMask)

    keyboard = cv2.waitKey(30)
    if keyboard == 'q' or keyboard == 27:
        break

cv2.waitKey(0)
cv2.destroyAllWindows()
capture.release()

### Спосіб 2: Optical Flow

In [2]:
import numpy as np
import cv2 as cv

cap = cv.VideoCapture(0)
ret, frame1 = cap.read()
prvs = cv.cvtColor(frame1,cv.COLOR_BGR2GRAY)
hsv = np.zeros_like(frame1)
hsv[..., 1] = 255

while True:
    ret, frame2 = cap.read()
    next = cv.cvtColor(frame2, cv.COLOR_BGR2GRAY)
    flow = cv.calcOpticalFlowFarneback(prvs, next, None, 0.5, 3, 15, 3, 5, 1.2, 0)
    mag, ang = cv.cartToPolar(flow[..., 0], flow[..., 1])
    hsv[..., 0] = ang*180/np.pi/2
    hsv[..., 2] = cv.normalize(mag, None, 0, 255, cv.NORM_MINMAX)
    bgr = cv.cvtColor(hsv, cv.COLOR_HSV2BGR)
    cv.imshow('frame2', bgr)
    
    keyboard = cv.waitKey(30)
    if keyboard == 'q' or keyboard == 27:
        break
    prvs = next

cv.waitKey(0)
cv.destroyAllWindows()
cap.release()

### Спосіб 3: Canny, findContours

[джерело](https://www.geeksforgeeks.org/find-and-draw-contours-using-opencv-python/)

In [17]:
import cv2 as cv
import numpy as np 
  
# Let's load a simple image with 3 black squares 
cap = cv.VideoCapture(0)
backSub = cv.createBackgroundSubtractorKNN(10, 1000, True)

while True:  
    ret, frame = cap.read()


    # Grayscale 
    gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY) 
    
    # Movement mask
    fgMask = backSub.apply(gray)

    # Find Canny edges 
    edged = cv.Canny(fgMask, 30, 200) 

    # Finding Contours 
    # Use a copy of the image e.g. edged.copy() 
    # since findContours alters the image 
    contours, hierarchy = cv.findContours(edged, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_NONE) 
    
    cv.imshow('Canny Edges After Contouring', edged)
  
    #print("Number of Contours found = " + str(len(contours))) 
  
    # Draw all contours 
    # -1 signifies drawing all contours 
    cv.drawContours(frame, contours, -1, (255, 255, 0), 3) 
    cv.imshow('frame', frame)

    if cv.waitKey(1) & 0xFF == ord('q'):
        break
  

cv.waitKey(0)
cv.destroyAllWindows()
cap.release()

### Спосіб 4

[джерело](https://stackoverflow.com/questions/41402137/opencv-splitting-contours)

In [37]:
import cv2 as cv

cap = cv.VideoCapture(0)
backSub = cv.createBackgroundSubtractorKNN(10, 1000, True)

while True:
    ret, frame = cap.read()
    image = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
    fgMask = backSub.apply(image)
    #image = cv.blur(fgMask, (10, 10))
    (_, img_th) = cv.threshold(fgMask, 150, 255, 1)
    (contours, _) = cv.findContours(img_th, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)

    cv.drawContours(fgMask, contours, -1, (255, 0, 0), 2)
    cv.imshow("contours", fgMask)

    cv.imshow("raw video", frame)

    if cv.waitKey(1) & 0xFF == ord('q'):
        break

cv.waitKey(0)
cv.destroyAllWindows()
cap.release()

### Спосіб 5: Optical Flow

[джерело](https://stackoverflow.com/questions/13685771/opencv-python-calcopticalflowfarneback)

Найбільш обнадійливий спосіб

In [54]:
from numpy import *
import cv2

def draw_flow(im, flow, step=16):
    h, w = im.shape[:2]
    y, x = mgrid[step/2:h:step, step/2:w:step].reshape(2, -1)
    fx, fy = flow[y, x].T

    # create line endpoints
    lines = vstack([x, y, x + fx, y + fy]).T.reshape(-1, 2, 2)
    lines = int32(lines)

    # create image and draw
    vis = cv2.cvtColor(im,cv2.COLOR_GRAY2BGR)
    for (x1,y1),(x2,y2) in lines:
        cv2.line(vis,(x1,y1),(x2,y2),(0,255,0),1)
        cv2.circle(vis,(x1,y1),1,(0,255,0), -1)
    return vis


cap = cv2.VideoCapture(0)

ret,im = cap.read()
prev_gray = cv2.cvtColor(im,cv2.COLOR_BGR2GRAY)

while True:
    # get grayscale image
    ret,im = cap.read()
    gray = cv2.cvtColor(im,cv2.COLOR_BGR2GRAY)

    # compute flow
    flow = cv2.calcOpticalFlowFarneback(prev_gray, gray, None, 0.5, 3, 15, 3, 5, 1.2, 0)
    prev_gray = gray

    # plot the flow vectors
    cv2.imshow('Optical flow', draw_flow(gray, flow))
    if cv2.waitKey(10) == 27:
        break

cv2.waitKey(0)
cv2.destroyAllWindows()
cap.release()

IndexError: arrays used as indices must be of integer (or boolean) type